****Processing Results****

In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from matplotlib import style
import re
import difflib as dl
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve

Finding counts of labelling outcomes

In [2]:
new = pd.read_csv('good_watson.csv') 

In [3]:
new

,gold,predicted,Unnamed: 2
0,"atypical, atypical ductal hyperplasia, na","atypical, atypical ductal hyperplasia, na",6
1,"atypical, atypical ductal hyperplasia, na","atypical, flat epithelial atypia, na",1
2,"atypical, atypical ductal hyperplasia, na","breast cancer, in situ, ductal",1
3,"atypical, atypical lobular hyperplasia, na","atypical, atypical lobular hyperplasia, na",14
4,"atypical, atypical lobular hyperplasia, na","atypical, phyllodes, na",1
5,"atypical, fibroadenoma, na","atypical, fibroadenoma, na",29
6,"atypical, flat epithelial atypia, na","atypical, atypical ductal hyperplasia, na",2
7,"atypical, flat epithelial atypia, na","atypical, flat epithelial atypia, na",2
8,"atypical, phyllodes, na","atypical, phyllodes, na",1
9,"benign, apocrine metaplasia, na","benign, apocrine metaplasia, na",9


Pivot graph

In [4]:
k = new.pivot(index='gold', columns='predicted')
k.fillna(0)


Unnamed: 2  \
predicted                                         atypical, atypical ductal hyperplasia, na   
gold                                                                                          
atypical, atypical ductal hyperplasia, na                                               6.0   
atypical, atypical lobular hyperplasia, na                                              0.0   
atypical, fibroadenoma, na                                                              0.0   
atypical, flat epithelial atypia, na                                                    2.0   
atypical, phyllodes, na                                                                 0.0   
benign, apocrine metaplasia, na                                                         0.0   
benign, cyst, na                                                                        0.0   
benign, mastitis, na                                                                    0.0   
benign, papilloma, na                                                                   0.0   
benign, pseudoangiomatous stromal hyperplasia, na                                       0.0   
benign, radial scar, na                                                                 0.0   
benign, sclerosing adenosis, na                                                         1.0   
benign, usual ductal hyperplasia, na                                                    0.0   
breast cancer, in situ, ductal                                                          0.0   
breast cancer, in situ, lobular                                                         0.0   
breast cancer, invasive, ductal                                                         0.0   
breast cancer, invasive, lobular                                                        0.0   
breast cancer, invasive, na                                                             0.0   
breast cancer, metastasis, na                                                           0.0   
breast cancer, na, ductal                                                               0.0   
breast cancer, na, lobular                                                              0.0   
breast cancer, na, na                                                                   0.0   

                                                                                              \
predicted                                         atypical, atypical lobular hyperplasia, na   
gold                                                                                           
atypical, atypical ductal hyperplasia, na                                                0.0   
atypical, atypical lobular hyperplasia, na                                              14.0   
atypical, fibroadenoma, na                                                               0.0   
atypical, flat epithelial atypia, na                                                     0.0   
atypical, phyllodes, na                                                                  0.0   
benign, apocrine metaplasia, na                                                          0.0   
benign, cyst, na                                                                         0.0   
benign, mastitis, na                                                                     0.0   
benign, papilloma, na                                                                    0.0   
benign, pseudoangiomatous stromal hyperplasia, na                                        0.0   
benign, radial scar, na                                                                  0.0   
benign, sclerosing adenosis, na                                                          0.0   
benign, usual ductal hyperplasia, na                                                     0.0   
breast cancer, in situ, ductal                                                           0.0   
breast cancer, in situ, lobular                                          

Processing of the data into a DataFrame

In [5]:
col_arr = np.array(k.index)
final_col_arr = np.append('lymphoma, na, na', col_arr)
final_col_arr = np.append('breast cancer, in situ, na', final_col_arr)
file = open("WatsonResultsMay2018.txt", 'r') 
sq = file.read() 
listing = re.findall(r"[\w\s]+, [\w\s]+, [\w\s]+.*?top_class.*?\[.*?\]", sq)
l = pd.DataFrame(np.zeros((len(listing),24)), columns=final_col_arr)
l['Single Label Derived'] = pd.Series("", index=l.index)
count = 0
for i in listing:
    actual = re.findall(r"[\w\s]+, [\w\s]+, [\w\s]+", i)[0]
    c_name = re.findall(r"{'class_name': '([\w\s]+, [\w\s]+, [\w\s]+)',", i)
    conf = re.findall(r"'confidence': ([\d.]+)}", i)
    l.at[count, 'Single Label Derived'] = actual
    for i in range(0, len(c_name)):
        l.at[count, c_name[i]] = conf[i]
    count += 1

In [6]:
l

watson_pred = pd.read_csv('watson_pred.csv', header=None)
watson_pred_arr = watson_pred.iloc[:, 0].values

In [7]:
l = l.replace('\n','', regex=True)
l['Scored Labels'] = watson_pred_arr
cols = list(l)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Single Label Derived')))
l = l.ix[:, cols]
conf_col(l, 'watson_final.csv')


NameError: name 'conf_col' is not defined

In [392]:
l = l.replace('\n','', regex=True)
l['Scored Labels'] = azure.iloc[:, 0]
cols = list(l)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Single Label Derived')))
l = l.ix[:, cols]
new_arr = np.zeros(len(l))
l['conf'] = new_arr
end = len(l.columns)-2
for i in range(0, len(l)):
    new_arr[i] =max(l.iloc[i].values[0:end].astype('float'))
l['conf'] = new_arr
l.to_csv('watson_with_conf.csv')
new_arr2 = np.zeros(len(l))
l['same'] = new_arr2
end = len(l.columns)-3
for k in range(0, len(l)):
    new_arr2[k] = l.iloc[k, 0].strip() == l.iloc[k, end].strip()
    l['same'] = new_arr2
l

ValueError: could not convert string to float: 'benign, cyst, na '

Plotting ROC Curves

In [27]:
new_arr = []
for i in l['actual label'].values:
    if "atypical, atypical ductal hyperplasia, na" in i:
        new_arr = np.append(new_arr, 1)
    else:
        new_arr = np.append(new_arr, 0)
new_arr
score = l["atypical, atypical ductal hyperplasia, na"].values
a, b ,c= roc_curve(new_arr, score)
plt.plot(a,b)

TypeError: argument of type 'float' is not iterable

Testing Purposes

In [9]:
tg = "breast cancer, invasive, ductal {'classes': 'breast cancer, invasive, ductal', 'text': '1   Microscopic focus of invasive ductal carcinoma, 0  4 mm   see comment   2   Ductal carcinoma in situ, high nuclear grade, mixed comedo, cribriform, micropapillary and solid types, spanning 10  5 cm, all excision margins free of carcinoma   see comment   3   Atypical lobular hyperplasia   4   Fibroadenoma   5   Fibrocystic changes   6   Stromal fibrosis   7   Two biopsy sites   8   Microcalcifications associated with ductal carcinoma in situ  '} {'classifier_id': '511c9dx382-nlc-383', 'url': 'https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/511c9dx382-nlc-383', 'text': '1   Microscopic focus of invasive ductal carcinoma, 0  4 mm   see comment   2   Ductal carcinoma in situ, high nuclear grade, mixed comedo, cribriform, micropapillary and solid types, spanning 10  5 cm, all excision margins free of carcinoma   see comment   3   Atypical lobular hyperplasia   4   Fibroadenoma   5   Fibrocystic changes   6   Stromal fibrosis   7   Two biopsy sites   8   Microcalcifications associated with ductal carcinoma in situ  ', 'top_class': 'breast cancer, invasive, ductal', 'classes': [{'class_name': 'breast cancer, invasive, ductal', 'confidence': 0.7687625985185428}, {'class_name': 'breast cancer, in situ, ductal', 'confidence': 0.17088211636450867}, {'class_name': 'breast cancer, invasive, lobular', 'confidence': 0.018307941501260583}, {'class_name': 'breast cancer, metastasis, na', 'confidence': 0.003874148425607778}, {'class_name': 'breast cancer, invasive, na', 'confidence': 0.0036593399731099396}, {'class_name': 'class', 'confidence': 0.003573795624038167}, {'class_name': 'breast cancer, na, na', 'confidence': 0.003241716302322546}, {'class_name': 'lymphoma, na, na', 'confidence': 0.003137873802075204}, {'class_name': 'breast cancer, na, lobular', 'confidence': 0.0030373577079201805}, {'class_name': 'breast cancer, in situ, na', 'confidence': 0.0010684211534858015}]}"
actual = re.findall(r"[\w\s]+, [\w\s]+, [\w\s]+", tg)[0]
#[{'class_name': 'breast cancer, in situ, ductal', 'confidence': 0.9769611826405123
actual

'breast cancer, invasive, ductal '

In [10]:
c_name = re.findall(r"{'class_name': '([\w\s]+, [\w\s]+, [\w\s]+)',", tg)
conf = re.findall(r"'confidence': ([\d.]+)}", tg)

In [11]:
conf

['0.7687625985185428',
 '0.17088211636450867',
 '0.018307941501260583',
 '0.003874148425607778',
 '0.0036593399731099396',
 '0.003573795624038167',
 '0.003241716302322546',
 '0.003137873802075204',
 '0.0030373577079201805',
 '0.0010684211534858015']

In [12]:
c_name

['breast cancer, invasive, ductal',
 'breast cancer, in situ, ductal',
 'breast cancer, invasive, lobular',
 'breast cancer, metastasis, na',
 'breast cancer, invasive, na',
 'breast cancer, na, na',
 'lymphoma, na, na',
 'breast cancer, na, lobular',
 'breast cancer, in situ, na']

In [13]:
l = pd.DataFrame(np.zeros((5,22)), columns=final_col_arr)
l['actual label'] = pd.Series("", index=l.index)
l.at[0, 'actual label'] = 'pizza!'
l


ValueError: Shape of passed values is (22, 5), indices imply (24, 5)

In [14]:
l.at[0, 'actual label'] = actual
for i in range(0, len(c_name)):
    l.at[0, c_name[i]] = conf[i]

In [15]:

final_col_arr

array(['breast cancer, in situ, na', 'lymphoma, na, na',
       'atypical, atypical ductal hyperplasia, na',
       'atypical, atypical lobular hyperplasia, na',
       'atypical, fibroadenoma, na',
       'atypical, flat epithelial atypia, na', 'atypical, phyllodes, na',
       'benign, apocrine metaplasia, na', 'benign, cyst, na',
       'benign, mastitis, na', 'benign, papilloma, na',
       'benign, pseudoangiomatous stromal hyperplasia, na',
       'benign, radial scar, na', 'benign, sclerosing adenosis, na',
       'benign, usual ductal hyperplasia, na',
       'breast cancer, in situ, ductal',
       'breast cancer, in situ, lobular',
       'breast cancer, invasive, ductal',
       'breast cancer, invasive, lobular', 'breast cancer, invasive, na',
       'breast cancer, metastasis, na', 'breast cancer, na, ductal',
       'breast cancer, na, lobular', 'breast cancer, na, na'],
      dtype=object)

In [16]:
l

,Single Label Derived,"breast cancer, in situ, na","lymphoma, na, na","atypical, atypical ductal hyperplasia, na","atypical, atypical lobular hyperplasia, na","atypical, fibroadenoma, na","atypical, flat epithelial atypia, na","atypical, phyllodes, na","benign, apocrine metaplasia, na","benign, cyst, na",...,"breast cancer, in situ, lobular","breast cancer, invasive, ductal","breast cancer, invasive, lobular","breast cancer, invasive, na","breast cancer, metastasis, na","breast cancer, na, ductal","breast cancer, na, lobular","breast cancer, na, na",Scored Labels,actual label
0,"benign, cyst, na",0.003037,0.003242,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.768763,0.018308,0.003659,0.003874,0.000000,0.003138,0.003574,"benign, cyst, na","breast cancer, invasive, ductal"
1,"atypical, fibroadenoma, na",0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"atypical, fibroadenoma, na",NaN
2,"breast cancer, invasive, ductal",0.003037,0.003242,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.768763,0.018308,0.003659,0.003874,0.000000,0.003138,0.003574,"breast cancer, invasive, ductal",NaN
3,"benign, cyst, na",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"benign, cyst, na",NaN
4,"benign, cyst, na",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"benign, cyst, na",NaN
5,"breast cancer, invasive, ductal",0.001047,0.001501,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.976941,0.000000,0.002046,0.004418,0.001491,0.000939,0.003828,"breast cancer, invasive, ductal",NaN
6,"breast cancer, na, na",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,"breast cancer, na, na",NaN
7,"atypical, flat epithelial atypia, na",0.004465,0.004031,0.000000,0.000000,0.000000,0.769922,0.174499,0.000000,0.000000,...,0.004941,0.000000,0.000000,0.000000,0.003327,0.005426,0.004570,0.000000,"atypical, flat epithelial atypia, na",NaN
8,"atypical, fibroadenoma, na",0.001085,0.001864,0.001658,0.002123,0.979645,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.001060,0.000000,0.000000,"atypical, fibroadenoma, na",NaN
9,"benign, radial scar, na",0.000000,0.005888,0.015346,0.037256,0.007751,0.000000,0.000000,0.296427,0.000000,...,0.006200,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"benign, radial scar, na",NaN


Processing Output from Azure Machine Learning Studio & calculating Accuracy

In [17]:

azure = pd.read_csv('azure_results_6.csv')
#azure = fix_in_file(azure, 25)
#azure = fix_in_file(azure, 0)
acc_azure_3 = azure.iloc[:, 0].values == azure.iloc[:, 25].values
acc_azure_3_calc = sum(acc_azure_3)/len(acc_azure_3)
acc_azure_3_calc
conf_col(azure, 'multiclass_decision_jungle_3.csv')

NameError: name 'conf_col' is not defined

In [18]:
azure.iloc[1, 0] != azure.iloc[1, 25]

False

In [19]:
#multiclass 3
acc_azure_3_calc

0.8485915492957746

In [20]:
#multiclass 1
acc_azure_3_calc

0.8485915492957746

In [21]:
azure_1_class = pd.read_csv('azure_results_17.csv')
acc_azure_1 = azure_1_class.iloc[:, 0].values == azure_1_class.iloc[:, 5].values
acc_azure_1_calc = sum(acc_azure_1)/len(acc_azure_1)
acc_azure_1_calc

0.9788732394366197

Display Sample for Sanity Checking

In [22]:
azure_1_class[azure_1_class.iloc[:, 0].values != azure_1_class.iloc[:, 5].values]

,Single Label Derived,"Scored Probabilities for Class ""atypical""","Scored Probabilities for Class ""benign""","Scored Probabilities for Class ""breast cancer""","Scored Probabilities for Class ""lymphoma""",Scored Labels
40,atypical,0.421055,0.015246,0.560331,0.003369,breast cancer
96,breast cancer,0.739581,0.027816,0.230438,0.002164,atypical
112,breast cancer,0.904365,0.000192,0.094857,0.000585,atypical
161,breast cancer,0.065808,0.894299,0.039647,0.000246,benign
162,atypical,0.243563,0.667605,0.084279,0.004553,benign
269,atypical,0.367824,0.033949,0.597000,0.001228,breast cancer


Helper Functions

In [23]:
def fix(csv, output_csv, row_num):
    df = pd.read_csv(csv)
    df = fix_in_file(df, row_num)
    df.to_csv(output_csv)
def fix_in_file(df, row_num):
    new_arr = []
    for k in df.iloc[:, row_num].values:
        new_arr = np.append(new_arr, re.findall(r"[\w\s]+", k)[0])
    df.iloc[:, row_num] = new_arr
    return df
def conf_col(df, csv_name):
    new_arr = np.zeros(len(df))
    new_arr2 = np.zeros(len(df))
    df['confidence'] = new_arr
    df['same'] = new_arr2
    end = len(df.columns)-3
    for k in range(0, len(df)):
        new_arr[k] =max(df.iloc[k].values[1:end].astype('float'))
        new_arr2[k] = df.iloc[k, 0].strip() == df.iloc[k, end].strip()
    df['confidence'] = new_arr
    df['same'] = new_arr2
    df.to_csv(csv_name)
        

In [24]:
fix('Training_Set.csv', "training_set_1.csv", 2)